# **Homework 2: Phoneme Classification**


Objectives:
* Solve a classification problem with deep neural networks (DNNs).
* Understand recursive neural networks (RNNs).

If you have any questions, please contact the TAs via TA hours, NTU COOL, or email to mlta-2023-spring@googlegroups.com

# Download Data
Download data from google drive, then unzip it.

You should have
- `libriphone/train_split.txt`: training metadata
- `libriphone/train_labels`: training labels
- `libriphone/test_split.txt`: testing metadata
- `libriphone/feat/train/*.pt`: training feature
- `libriphone/feat/test/*.pt`:  testing feature

after running the following block.

> **Notes: if the google drive link is dead, you can download the data directly from [Kaggle](https://www.kaggle.com/c/ml2023spring-hw2/data) and upload it to the workspace.**


In [1]:
# !pip install --upgrade gdown

# # Main link
# # !gdown --id '1N1eVIDe9hKM5uiNRGmifBlwSDGiVXPJe' --output libriphone.zip
# !gdown --id '1qzCRnywKh30mTbWUEjXuNT2isOCAPdO1' --output libriphone.zip

# !unzip -q libriphone.zip
# !ls libriphone

# Some Utility Functions
**Fixes random number generator seeds for reproducibility.**

In [2]:
import numpy as np
import torch
import random
import time

from torchinfo import summary
from torch.utils.tensorboard import SummaryWriter


def same_seeds(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

**Helper functions to pre-process the training data from raw MFCC features of each utterance.**

A phoneme may span several frames and is dependent to past and future frames. \
Hence we concatenate neighboring phonemes for training to achieve higher accuracy. The **concat_feat** function concatenates past and future k frames (total 2k+1 = n frames), and we predict the center frame.

Feel free to modify the data preprocess functions, but **do not drop any frame** (if you modify the functions, remember to check that the number of frames are the same as mentioned in the slides)

In [3]:
import os
import torch
from tqdm import tqdm


def load_feat(path):
    feat = torch.load(path)
    return feat


def shift(x, n):
    if n < 0:
        left = x[0].repeat(-n, 1)
        right = x[:n]
    elif n > 0:
        right = x[-1].repeat(n, 1)
        left = x[n:]
    else:
        return x

    return torch.cat((left, right), dim=0)


def concat_feat(x, concat_n):
    assert concat_n % 2 == 1  # n must be odd
    if concat_n < 2:
        return x
    seq_len, feature_dim = x.size(0), x.size(1)
    x = x.repeat(1, concat_n)
    x = x.view(seq_len, concat_n, feature_dim).permute(
        1, 0, 2)  # concat_n, seq_len, feature_dim
    mid = (concat_n // 2)
    for r_idx in range(1, mid+1):
        x[mid + r_idx, :] = shift(x[mid + r_idx], r_idx)
        x[mid - r_idx, :] = shift(x[mid - r_idx], -r_idx)

    return x.permute(1, 0, 2).view(seq_len, concat_n * feature_dim)


def preprocess_data(split, feat_dir, phone_path, concat_nframes, train_ratio=0.8, random_seed=1213):
    class_num = 41  # NOTE: pre-computed, should not need change

    if split == 'train' or split == 'val':
        mode = 'train'
    elif split == 'test':
        mode = 'test'
    else:
        raise ValueError(
            'Invalid \'split\' argument for dataset: PhoneDataset!')

    label_dict = {}
    if mode == 'train':
        for line in open(os.path.join(phone_path, f'{mode}_labels.txt')).readlines():
            line = line.strip('\n').split(' ')
            label_dict[line[0]] = [int(p) for p in line[1:]]

        # split training and validation data
        usage_list = open(os.path.join(
            phone_path, 'train_split.txt')).readlines()
        random.seed(random_seed)
        random.shuffle(usage_list)
        train_len = int(len(usage_list) * train_ratio)
        usage_list = usage_list[:train_len] if split == 'train' else usage_list[train_len:]

    elif mode == 'test':
        usage_list = open(os.path.join(
            phone_path, 'test_split.txt')).readlines()

    usage_list = [line.strip('\n') for line in usage_list]
    print('[Dataset] - # phone classes: ' + str(class_num) +
          ', number of utterances for ' + split + ': ' + str(len(usage_list)))

    max_len = 3000000
    X = torch.empty(max_len, 39 * concat_nframes)
    if mode == 'train':
        y = torch.empty(max_len, dtype=torch.long)

    idx = 0
    for i, fname in tqdm(enumerate(usage_list)):
        feat = load_feat(os.path.join(feat_dir, mode, f'{fname}.pt'))
        cur_len = len(feat)
        feat = concat_feat(feat, concat_nframes)
        if mode == 'train':
            label = torch.LongTensor(label_dict[fname])

        X[idx: idx + cur_len, :] = feat
        if mode == 'train':
            y[idx: idx + cur_len] = label

        idx += cur_len

    X = X[:idx, :]
    if mode == 'train':
        y = y[:idx]

    print(f'[INFO] {split} set')
    print(X.shape)
    if mode == 'train':
        print(y.shape)
        return X, y
    else:
        return X

# Dataset

In [4]:
import torch
from torch.utils.data import Dataset


class LibriDataset(Dataset):
    def __init__(self, X, y=None):
        self.data = X
        if y is not None:
            self.label = torch.LongTensor(y)
        else:
            self.label = None

    def __getitem__(self, idx):
        if self.label is not None:
            return self.data[idx], self.label[idx]
        else:
            return self.data[idx]

    def __len__(self):
        return len(self.data)

# Model
Feel free to modify the structure of the model.

In [5]:
import torch.nn as nn


class BasicBlock(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(BasicBlock, self).__init__()

        # TODO: apply batch normalization and dropout for strong baseline.
        # Reference: https://pytorch.org/docs/stable/generated/torch.nn.BatchNorm1d.html (batch normalization)
        #       https://pytorch.org/docs/stable/generated/torch.nn.Dropout.html (dropout)
        self.block = nn.Sequential(
            nn.Linear(input_dim, output_dim),
            nn.ReLU(),
        )

    def forward(self, x):
        x = self.block(x)
        return x


class Classifier(nn.Module):
    def __init__(self, input_dim, output_dim=41, hidden_layers=1, hidden_dim=256):
        super(Classifier, self).__init__()

        self.fc = nn.Sequential(
            BasicBlock(input_dim, hidden_dim),
            *[BasicBlock(hidden_dim, hidden_dim)
              for _ in range(hidden_layers)],
            nn.Linear(hidden_dim, output_dim)
        )

    def forward(self, x):
        x = self.fc(x)
        return x


class BiLSTMClassifier(nn.Module):
    def __init__(self, input_dim=39, output_dim=41, hidden_layers=3, hidden_dim=356, dropout=0.5, seq_length=19):
        super().__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.seq_length = seq_length
        self.lstm = nn.LSTM(input_dim, hidden_dim, dropout=dropout,
                            num_layers=hidden_layers, bidirectional=True, batch_first=True)
        self.fc = nn.Sequential(
            nn.BatchNorm1d(hidden_dim * 2),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim * 2, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.Dropout(dropout),
            nn.LeakyReLU(),
            nn.Linear(hidden_dim, output_dim),
        )

    def forward(self, x):
        batch, _, = x.shape
        x = x.view(batch, -1, self.input_dim)
        x, _ = self.lstm(x)
        x = x[:, self.seq_length // 2]
        x = self.fc(x)
        return x

# Hyper-parameters

In [6]:
# data prarameters
# TODO: change the value of "concat_nframes" for medium baseline
# the number of frames to concat with, n must be odd (total 2k+1 = n frames)
concat_nframes = 39
# the ratio of data used for training, the rest will be used for validation
train_ratio = 0.90

# training parameters
seed = 1314          # random seed
batch_size = 1024        # batch size
num_epoch = 15         # the number of training epoch
learning_rate = 1e-4      # learning rate
model_path = './model.ckpt'  # the path where the checkpoint will be saved

# model parameters
# TODO: change the value of "hidden_layers" or "hidden_dim" for medium baseline
# the input dim of the model, you should not change the value
# input_dim = 39 * concat_nframes
input_dim = 39
hidden_layers = 4          # the number of hidden layers
hidden_dim = 356          # the hidden dim

# Dataloader

In [7]:
from torch.utils.data import DataLoader
import gc

same_seeds(seed)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'DEVICE: {device}')

# preprocess data
train_X, train_y = preprocess_data(split='train', feat_dir='./libriphone/feat', phone_path='./libriphone',
                                   concat_nframes=concat_nframes, train_ratio=train_ratio, random_seed=seed)
val_X, val_y = preprocess_data(split='val', feat_dir='./libriphone/feat', phone_path='./libriphone',
                               concat_nframes=concat_nframes, train_ratio=train_ratio, random_seed=seed)

# get dataset
train_set = LibriDataset(train_X, train_y)
val_set = LibriDataset(val_X, val_y)

# remove raw feature to save memory
del train_X, train_y, val_X, val_y
gc.collect()

# get dataloader
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False)

DEVICE: cuda
[Dataset] - # phone classes: 41, number of utterances for train: 3086


3086it [00:44, 69.74it/s]


[INFO] train set
torch.Size([1901590, 1521])
torch.Size([1901590])
[Dataset] - # phone classes: 41, number of utterances for val: 343


343it [00:04, 71.15it/s]

[INFO] val set
torch.Size([215204, 1521])
torch.Size([215204])


# Training

In [8]:
# create model, define a loss function, and optimizer
# model = Classifier(input_dim=input_dim, hidden_layers=hidden_layers,
#                    hidden_dim=hidden_dim).to(device)
model = BiLSTMClassifier(
    input_dim=input_dim, hidden_dim=hidden_dim, hidden_layers=hidden_layers, seq_length=concat_nframes).to(device)
summary(model, input_size=(batch_size, concat_nframes * input_dim))

Layer (type:depth-idx)                   Output Shape              Param #
BiLSTMClassifier                         [1024, 41]                --
├─LSTM: 1-1                              [1024, 39, 896]           11,400,704
├─Sequential: 1-2                        [1024, 41]                --
│    └─Linear: 2-1                       [1024, 448]               401,856
│    └─BatchNorm1d: 2-2                  [1024, 448]               896
│    └─Dropout: 2-3                      [1024, 448]               --
│    └─LeakyReLU: 2-4                    [1024, 448]               --
│    └─Linear: 2-5                       [1024, 41]                18,409
Total params: 11,821,865
Trainable params: 11,821,865
Non-trainable params: 0
Total mult-adds (G): 455.73
Input size (MB): 6.23
Forward/backward pass size (MB): 293.94
Params size (MB): 47.29
Estimated Total Size (MB): 347.45

In [9]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(
    model.parameters(), lr=learning_rate, weight_decay=1e-2)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(
    optimizer, 5, 1, learning_rate/100)

writer = SummaryWriter(f"runs/{time.strftime('%Y-%m-%d_%H-%M-%S')}")

best_acc = 0.0
early_stop_count = 0
for epoch in range(num_epoch):
    train_acc = 0.0
    train_loss = 0.0
    val_acc = 0.0
    val_loss = 0.0

    # training
    model.train()  # set the model to training mode
    for i, batch in enumerate(tqdm(train_loader)):
        features, labels = batch
        features = features.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(features)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # get the index of the class with the highest probability
        _, train_pred = torch.max(outputs, 1)
        train_acc += (train_pred.detach() == labels.detach()).sum().item()
        train_loss += loss.item()

        writer.add_scalar('Loss/Train', loss.item(),
                          len(train_loader) * epoch + i)
    writer.add_scalar('Acc/Train', train_acc/len(train_set), epoch)
    scheduler.step()

    # validation
    model.eval()  # set the model to evaluation mode
    with torch.no_grad():
        for i, batch in enumerate(tqdm(val_loader)):
            features, labels = batch
            features = features.to(device)
            labels = labels.to(device)
            outputs = model(features)

            loss = criterion(outputs, labels)

            _, val_pred = torch.max(outputs, 1)
            # get the index of the class with the highest probability
            val_acc += (val_pred.cpu() == labels.cpu()).sum().item()
            val_loss += loss.item()
    writer.add_scalar('Loss/val', val_loss/len(val_loader), epoch)
    writer.add_scalar('Acc/Val', val_acc/len(val_set), epoch)

    print(f'[{epoch+1:03d}/{num_epoch:03d}] Train Acc: {train_acc/len(train_set):3.5f} Loss: {train_loss/len(train_loader):3.5f} | Val Acc: {val_acc/len(val_set):3.5f} loss: {val_loss/len(val_loader):3.5f}')

    # if the model improves, save a checkpoint at this epoch
    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(model.state_dict(), model_path)
        print(f'saving model with acc {best_acc/len(val_set):.5f}')
        early_stop_count = 0
    else:
        early_stop_count += 1
    if early_stop_count >= 3:
        print('\nModel is not improving, so we halt the training session.')
        writer.close()
        break


100%|██████████| 211/211 [00:12<00:00, 17.48it/s]


[001/030] Train Acc: 0.62888 Loss: 1.25794 | Val Acc: 0.73379 loss: 0.85846
saving model with acc 0.73379


100%|██████████| 211/211 [00:07<00:00, 29.73it/s]


[002/030] Train Acc: 0.77475 Loss: 0.72694 | Val Acc: 0.77048 loss: 0.74862
saving model with acc 0.77048


100%|██████████| 211/211 [00:07<00:00, 29.93it/s]


...

[006/030] Train Acc: 0.85974 Loss: 0.43248 | Val Acc: 0.77815 loss: 0.81234


100%|██████████| 211/211 [00:07<00:00, 28.30it/s]


[007/030] Train Acc: 0.88174 Loss: 0.35673 | Val Acc: 0.78040 loss: 0.85671


100%|██████████| 211/211 [00:07<00:00, 27.23it/s]

[008/030] Train Acc: 0.89941 Loss: 0.29476 | Val Acc: 0.78270 loss: 0.90785

Model is not improving, so we halt the training session.


In [10]:
del train_set, val_set
del train_loader, val_loader
gc.collect()

0

# Testing
Create a testing dataset, and load model from the saved checkpoint.

In [11]:
# load data
test_X = preprocess_data(split='test', feat_dir='./libriphone/feat',
                         phone_path='./libriphone', concat_nframes=concat_nframes)
test_set = LibriDataset(test_X, None)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)

[Dataset] - # phone classes: 41, number of utterances for test: 857


857it [00:12, 69.04it/s]

[INFO] test set
torch.Size([527364, 1521])


In [12]:
# load model
# model = Classifier(input_dim=input_dim, hidden_layers=hidden_layers,
#                    hidden_dim=hidden_dim).to(device)
model = BiLSTMClassifier(
    input_dim=input_dim, hidden_dim=hidden_dim, hidden_layers=hidden_layers, seq_length=concat_nframes).to(device)
model.load_state_dict(torch.load(model_path))

<All keys matched successfully>

Make prediction.

In [13]:
pred = np.array([], dtype=np.int32)

model.eval()
with torch.no_grad():
    for i, batch in enumerate(tqdm(test_loader)):
        features = batch
        features = features.to(device)

        outputs = model(features)

        # get the index of the class with the highest probability
        _, test_pred = torch.max(outputs, 1)
        pred = np.concatenate((pred, test_pred.cpu().numpy()), axis=0)

100%|██████████| 516/516 [00:17<00:00, 29.72it/s]


Write prediction to a CSV file.

After finish running this block, download the file `prediction.csv` from the files section on the left-hand side and submit it to Kaggle.

In [14]:
with open('prediction.csv', 'w') as f:
    f.write('Id,Class\n')
    for i, y in enumerate(pred):
        f.write('{},{}\n'.format(i, y))